In [106]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.2f}'.format
data = pd.read_csv(r'C:\Users\makos2tamas911\Documents\dev\mm\k_monitor_visu\data_cleaning\cleaned_data.csv')
data.head(1)

,Eljárás EKR azonosító,Ajánlatkérő szervezet neve,Ajánlatkérő nemzeti azonosítószáma,Ajánlatkérő szervezet típusa,Ajánlatkérő szervezet főtevékenysége,Eljárásrend,Eljárás fajtája,Eljárás tárgya,Szerződés típusa,A beszerzés végleges összértéke,A beszerzés végleges összértéke pénznem,Teljesítés helye NUTS-kód(ok),Szerződés megkötésének dátuma,Beérkezett ajánlatok száma,Nyertes ajánlattevő adószáma (adóazonosító jele),Nyertes ajánlattevő neve,Fő CPV-kód(ok),A beszerzés európai uniós alapokból finanszírozott projekttel és/vagy programmal kapcsolatos,Szerződés/rész odaítélésre került,Nyertes ajánlattevő kkv,Hirdetmény közzétételének dátuma,Minőségi kritérium alkalmazásra került,Költség kritérium alkalmazásra került,Ár kritérium alkalmazásra került,Ajánlatkérő szervezet neve - tisztított,Ajánlatkérő Cégtípus,Nyertes ajánlattevő neve - tisztított,Nyertes Cégtípus,unique_id,Hirdetmény és szerződés megkötés között eltelt idő
0,EKR001530432019,Albert Schweitzer Kórház Rendelőintézet,15813774210,U - Egyéb típus:,KL - Egészségügy,Uniós eljárásrend,Nyílt eljárás,Traumatológiai implantátumok beszerzése,Árubeszerzés,110810613.00,HUF,HU312,2020-07-14,1.00,"11172790210,HU11172790",Sanatmetal Ortopédiai És Traumatológiai Eszköz...,33184100-4,Nem,Igen,Igen,2020-07-23,Nem,Nem,Igen,Albert Schweitzer Kórház Rendelőintézet,NaN,Sanatmetal Ortopédiai És Traumatológiai Eszköz...,Kft.,0,-9 days


In [107]:
nan_counts = data.isna().sum()
nan_summary = nan_counts[nan_counts > 0]
print(nan_summary)
data = data.dropna(subset=['A beszerzés végleges összértéke'])

Ajánlatkérő szervezet típusa                                                                     5519
Eljárás tárgya                                                                                      6
Szerződés típusa                                                                                  153
A beszerzés végleges összértéke                                                                   779
A beszerzés végleges összértéke pénznem                                                           687
Teljesítés helye NUTS-kód(ok)                                                                     917
Szerződés megkötésének dátuma                                                                     195
Beérkezett ajánlatok száma                                                                          1
Nyertes ajánlattevő adószáma (adóazonosító jele)                                                 2526
Fő CPV-kód(ok)                                                                    

In [108]:
data.shape

(98396, 30)

In [109]:
unique_columns = {col: data[col].is_unique for col in data.columns}
unique_columns_df = {col: unique_columns[col] for col in unique_columns if unique_columns[col]}
print(unique_columns_df)

{'unique_id': True}


In [110]:
data = data.drop(['unique_id'], axis=1)

before = data.shape[0]
data = data.drop_duplicates()
after = data.shape[0]
print('Number of duplicate records dropped:', before-after)

Number of duplicate records dropped: 20046


In [111]:
unique_columns = {col: data[col].is_unique for col in data.columns}
unique_columns_df = {col: unique_columns[col] for col in unique_columns if unique_columns[col]}
print(unique_columns_df)

{}


In [112]:
data = data.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [113]:
inconsistent_ajanlatevok = data.groupby('Eljárás EKR azonosító')['Ajánlatkérő szervezet neve - tisztított'].nunique()
inconsistent_osszertek = data.groupby('Eljárás EKR azonosító')['A beszerzés végleges összértéke'].nunique()
inconsistent_ajanlatevok = inconsistent_ajanlatevok[inconsistent_ajanlatevok > 1]
inconsistent_osszertek = inconsistent_osszertek[inconsistent_osszertek > 1]

print(inconsistent_ajanlatevok.shape)
print(inconsistent_osszertek.shape)

(1828,)
(313,)


In [114]:
inconsistent_ajanlatevok.head()

Eljárás EKR azonosító
EKR000001102020    2
EKR000001722021    2
EKR000003232020    2
EKR000003542021    2
EKR000003782019    2
Name: Ajánlatkérő szervezet neve - tisztított, dtype: int64

##### 1828 esetben van több ajánlatkérő egy EKR azonosítóra
- ez valószínűleg lehetséges
##### 313 esetben van több összérték egy EKR azonosítóra
- ez valószínűleg adathiba, így dobom ezeket a sorokat

In [115]:
# példa az egyikre
data[data['Eljárás EKR azonosító'] == 'EKR000003542021'][['Ajánlatkérő szervezet neve', 'A beszerzés végleges összértéke']].value_counts()

Ajánlatkérő szervezet neve                                                     A beszerzés végleges összértéke
Nkk Nemzeti Közlekedési Központ Nonprofit Zártkörűen Működő Részvénytársaság   2342200.00                         2
Bfk Budapest Fejlesztési Központ Nonprofit Zártkörűen Működő Részvénytársaság  2430400.00                         1
Nkk Nemzeti Közlekedési Központ Nonprofit Zártkörűen Működő Részvénytársaság   3488800.00                         1
                                                                               30000000.00                        1
Name: count, dtype: int64

In [116]:
data = data[~data['Eljárás EKR azonosító'].isin(inconsistent_ajanlatevok.index)]

#### Szerződés Típus szerint 15 legtöbb ajánlatot kiíró és szerződést nyerő cég
- Árubeszerzés (szerződés típusa)
- Építési beruházás (szerződés típusa)
- Szolgáltatás (szerződés típusa)

In [117]:
data['Szerződés típusa'].value_counts().sort_values(ascending=False)

Szerződés típusa
Árubeszerzés                25666
Építési beruházás           17648
Szolgáltatás megrendelés    15796
Name: count, dtype: int64

In [118]:
kiíró = data.groupby(['Ajánlatkérő szervezet neve - tisztított', 'Szerződés típusa']).size().reset_index(name='count')

kiíró = kiíró.sort_values(by='count', ascending=False)

top_15_arubeszerzes_kiiro = kiíró[kiíró['Szerződés típusa'] == 'Árubeszerzés'].head(15)
top_15_szolgaltatas_kiiro = kiíró[kiíró['Szerződés típusa'] == 'Szolgáltatás megrendelés'].head(15)
top_15_epitesi_kiiro = kiíró[kiíró['Szerződés típusa'] == 'Építési beruházás'].head(15)

In [119]:
top_15_arubeszerzes_kiiro

,Ajánlatkérő szervezet neve - tisztított,Szerződés típusa,count
5323,Szegedi Tudományegyetem,Árubeszerzés,1252
1216,Debreceni Egyetem,Árubeszerzés,1061
4844,Pécsi Tudományegyetem,Árubeszerzés,939
1281,Digitális Kormányzati Ügynökség Zrt.,Árubeszerzés,710
5045,Semmelweis Egyetem,Árubeszerzés,675
810,Budapesti Közlekedési Zrt.,Árubeszerzés,548
882,Bácskiskun Megyei Kórház A Szegedi Tudományegy...,Árubeszerzés,529
3462,Magyar Közút Nonprofit Zrt.,Árubeszerzés,450
612,Borsodabaújzemplén Megyei Központi Kórház És E...,Árubeszerzés,421
3437,Magyar Honvédség Egészségügyi Központ,Árubeszerzés,286


In [120]:
top_15_arubeszerzes_kiiro = data[(data['Ajánlatkérő szervezet neve - tisztított'].isin(top_15_arubeszerzes_kiiro['Ajánlatkérő szervezet neve - tisztított']) & (data['Szerződés típusa'] == 'Árubeszerzés'))]
top_15_arubeszerzes_kiiro.to_csv(r'C:\Users\makos2tamas911\Documents\dev\mm\k_monitor_visu\data_exports\top_15_arubeszerzes_kiiro.csv', index=False)
print(top_15_arubeszerzes_kiiro.shape)

(8162, 29)


In [121]:
top_15_szolgaltatas_kiiro

,Ajánlatkérő szervezet neve - tisztított,Szerződés típusa,count
4190,Nemzeti Kommunikációs Hivatal,Szolgáltatás megrendelés,1108
3461,Magyar Közút Nonprofit Zrt.,Szolgáltatás megrendelés,980
4278,Nif Nemzeti Infrastruktúra Fejlesztő Zrt.,Szolgáltatás megrendelés,530
1280,Digitális Kormányzati Ügynökség Zrt.,Szolgáltatás megrendelés,506
2486,Innovációs És Technológiai Minisztérium,Szolgáltatás megrendelés,379
6273,Volánbusz Közlekedési Zrt.,Szolgáltatás megrendelés,354
809,Budapesti Közlekedési Zrt.,Szolgáltatás megrendelés,309
4263,Nhkv Nemzeti Hulladékgazdálkodási Koordináló É...,Szolgáltatás megrendelés,283
3430,Magyar Földgáztároló Zrt.,Szolgáltatás megrendelés,280
3484,Magyar Nemzeti Bank,Szolgáltatás megrendelés,197


In [122]:
top_15_szolgaltatas_kiiro = data[(data['Ajánlatkérő szervezet neve - tisztított'].isin(top_15_szolgaltatas_kiiro['Ajánlatkérő szervezet neve - tisztított'])) & (data['Szerződés típusa'] == 'Szolgáltatás megrendelés')]
top_15_szolgaltatas_kiiro.to_csv(r'C:\Users\makos2tamas911\Documents\dev\mm\k_monitor_visu\data_exports\top_15_szolgaltatas_kiiro.csv', index=False)
print(top_15_szolgaltatas_kiiro.shape)

(5580, 29)


In [123]:
top_15_epitesi_kiiro

,Ajánlatkérő szervezet neve - tisztított,Szerződés típusa,count
3463,Magyar Közút Nonprofit Zrt.,Építési beruházás,445
547,"Bmsk Beruházási, Műszaki Fejlesztési, Sportüze...",Építési beruházás,250
4280,Nif Nemzeti Infrastruktúra Fejlesztő Zrt.,Építési beruházás,187
1213,Debrecen Megyei Jogú Város Önkormányzata,Építési beruházás,157
414,Bdk Budapesti Dísz És Közvilágítási Kft.,Építési beruházás,123
749,Budapest Főváros Xvi. Kerületi Önkormányzat,Építési beruházás,112
4356,Nyíregyháza Megyei Jogú Város Önkormányzata,Építési beruházás,106
5600,Székesfehérvár Megyei Jogú Város Önkormányzata,Építési beruházás,98
695,Budapest Főváros Vagyonkezelő Központ Zrt.,Építési beruházás,95
4300,Nkm Áramhálózati Kft.,Építési beruházás,93


In [124]:
top_15_epitesi_kiiro = data[(data['Ajánlatkérő szervezet neve - tisztított'].isin(top_15_epitesi_kiiro['Ajánlatkérő szervezet neve - tisztított'])) & (data['Szerződés típusa'] == 'Építési beruházás')]
top_15_epitesi_kiiro.to_csv(r'C:\Users\makos2tamas911\Documents\dev\mm\k_monitor_visu\data_exports\top_15_epitesi_kiiro.csv', index=False)
print(top_15_epitesi_kiiro.shape)

(2072, 29)


In [125]:
data['procurement_id'] = data['Eljárás EKR azonosító'] + '_' + data['Nyertes ajánlattevő neve - tisztított']

nyertes = data.groupby(['Nyertes ajánlattevő neve - tisztított', 'Szerződés típusa'])['procurement_id'].nunique().reset_index(name='count')

nyertes = nyertes.sort_values(by='count', ascending=False)

top_15_arubeszerzes_nyertes = nyertes[nyertes['Szerződés típusa'] == 'Árubeszerzés'].head(15)
top_15_szolgaltatas_nyertes = nyertes[nyertes['Szerződés típusa'] == 'Szolgáltatás megrendelés'].head(15)
top_15_epitesi_nyertes = nyertes[nyertes['Szerződés típusa'] == 'Építési beruházás'].head(15)

In [126]:
top_15_arubeszerzes_nyertes

,Nyertes ajánlattevő neve - tisztított,Szerződés típusa,count
3143,Etiam Kft.,Árubeszerzés,184
7736,Nkm Energia Zrt.,Árubeszerzés,137
7422,Mvm Next Energiakereskedelmi Zrt.,Árubeszerzés,130
382,"Allegro , Kivitelező És Kft.",Árubeszerzés,122
9196,Rk Tech Kft.,Árubeszerzés,122
943,Balázsdiák Kft.,Árubeszerzés,116
7299,Mortak Fruit Kft.,Árubeszerzés,114
861,B. Braun Trading Kft.,Árubeszerzés,113
5352,Johnson Johnson Egészségügyi És Babaápolási Te...,Árubeszerzés,109
725,Atlas Pharma Gyógyszernagykereskedelmi Kft.,Árubeszerzés,105


In [127]:
top_15_arubeszerzes_nyertes = data[(data['Nyertes ajánlattevő neve - tisztított'].isin(top_15_arubeszerzes_nyertes['Nyertes ajánlattevő neve - tisztított'])) & (data['Szerződés típusa'] == 'Árubeszerzés')]
top_15_arubeszerzes_nyertes.to_csv(r'C:\Users\makos2tamas911\Documents\dev\mm\k_monitor_visu\data_exports\top_15_arubeszerzes_nyertes.csv', index=False)
print(top_15_arubeszerzes_nyertes.shape)

(2511, 30)


In [128]:
top_15_szolgaltatas_nyertes

,Nyertes ajánlattevő neve - tisztított,Szerződés típusa,count
7674,New Land Media Reklám Kft.,Szolgáltatás megrendelés,186
6426,Lounge Design Kft.,Szolgáltatás megrendelés,165
11208,Utiber Közúti Beruházó Kft.,Szolgáltatás megrendelés,81
10983,Tsystems Magyarország Zrt.,Szolgáltatás megrendelés,74
508,Antenna Hungária Magyar Műsorszóró És Rádióhír...,Szolgáltatás megrendelés,72
11065,Tömb 2002 Kft.,Szolgáltatás megrendelés,69
3765,Főmterv Mérnöki Tervező Zrt.,Szolgáltatás megrendelés,67
384,Allianz Hungária Zrt.,Szolgáltatás megrendelés,60
3912,Generali Biztosító Zrt.,Szolgáltatás megrendelés,54
1324,Bn Referencia Ipari Zrt.,Szolgáltatás megrendelés,53


In [129]:
top_15_szolgaltatas_nyertes = data[(data['Nyertes ajánlattevő neve - tisztított'].isin(top_15_szolgaltatas_nyertes['Nyertes ajánlattevő neve - tisztított'])) & (data['Szerződés típusa'] == 'Szolgáltatás megrendelés')]
top_15_szolgaltatas_nyertes.to_csv(r'C:\Users\makos2tamas911\Documents\dev\mm\k_monitor_visu\data_exports\top_15_szolgaltatas_nyertes.csv', index=False)
print(top_15_szolgaltatas_nyertes.shape)

(1801, 30)


In [130]:
top_15_epitesi_nyertes

,Nyertes ajánlattevő neve - tisztított,Szerződés típusa,count
9993,Strabag Általános Építő Kft.,Építési beruházás,135
11067,Tömb 2002 Kft.,Építési beruházás,109
1894,Colas Út Építőipari Zrt.,Építési beruházás,99
8343,Penta Általános Építőipari Kft.,Építési beruházás,93
10069,Swietelsky Magyarország Kft.,Építési beruházás,91
5636,Kevíz 21 Építőipari Zrt.,Építési beruházás,62
11466,Vianova 87 Közmű És Útépítő Zrt.,Építési beruházás,53
12212,Útéppark Útépítő És Mélyépítő Kft.,Építési beruházás,50
3702,Futizo Kft.,Építési beruházás,43
4702,Horváth Építőmester Építőipari Zrt.,Építési beruházás,43


In [131]:
top_15_epitesi_nyertes = data[(data['Nyertes ajánlattevő neve - tisztított'].isin(top_15_epitesi_nyertes['Nyertes ajánlattevő neve - tisztított'])) & (data['Szerződés típusa'] == 'Építési beruházás')]
top_15_epitesi_nyertes.to_csv(r'C:\Users\makos2tamas911\Documents\dev\mm\k_monitor_visu\data_exports\top_15_epitesi_nyertes.csv', index=False)
print(top_15_epitesi_nyertes.shape)

(1263, 30)


#### Szerződés összértéke szerint 5 legtöbb ajánlatot kiíró és szerződést nyerő cég

- 800M összérték feletti építési beruházások csoportja
- 300M összérték alatti építési beruházások csoportja
- 100M összérték alatti építési beruházások csoportja
- 900M összérték fölötti, de 1 ajánlattevős kiírások csoportja
- 300M összérték alatti, de 1 ajánlattevős kiírások csoportja

In [132]:
grouped_data_800 = data[data['Szerződés típusa'] == 'Építési beruházás'].groupby('Eljárás EKR azonosító')['A beszerzés végleges összértéke'].max().reset_index()
epitesi_osszertek_800M = grouped_data_800[grouped_data_800['A beszerzés végleges összértéke'] > 800000000]
print(epitesi_osszertek_800M.shape, '\n')

epitesi_osszertek_800M_full = data[data['Eljárás EKR azonosító'].isin(epitesi_osszertek_800M['Eljárás EKR azonosító'])]
top_5_kiiro_800 = epitesi_osszertek_800M_full['Ajánlatkérő szervezet neve - tisztított'].value_counts().head(5)
top_5_nyertes_800 = epitesi_osszertek_800M_full['Nyertes ajánlattevő neve - tisztított'].value_counts().head(5)

top_5_kiiro_800 = data[(data['Ajánlatkérő szervezet neve - tisztított'].isin(top_5_kiiro_800.index)) & (data['Szerződés típusa'] == 'Építési beruházás')]
top_5_nyertes_800 = data[(data['Nyertes ajánlattevő neve - tisztított'].isin(top_5_nyertes_800.index)) & (data['Szerződés típusa'] == 'Építési beruházás')]
top_5_kiiro_800.to_csv(r'C:\Users\makos2tamas911\Documents\dev\mm\k_monitor_visu\data_exports\top_5_kiiro_800M_epitesi.csv', index=False)
top_5_nyertes_800.to_csv(r'C:\Users\makos2tamas911\Documents\dev\mm\k_monitor_visu\data_exports\top_5_nyertes_800M_epitesi.csv', index=False)

print(top_5_kiiro_800)
print('\n')
print(top_5_nyertes_800)

(987, 2) 

      Eljárás EKR azonosító  \
2           EKR001531392019   
3           EKR001531392019   
25          EKR001534692019   
27          EKR001534762019   
39          EKR001536652019   
...                     ...   
98742       EKR000901422019   
98744       EKR000901422019   
98751       EKR000901422019   
98894       EKR000989822019   
98939       EKR001505382019   

                              Ajánlatkérő szervezet neve  \
2      Bmsk Beruházási, Műszaki Fejlesztési, Sportüze...   
3      Bmsk Beruházási, Műszaki Fejlesztési, Sportüze...   
25     Bmsk Beruházási, Műszaki Fejlesztési, Sportüze...   
27     Bmsk Beruházási, Műszaki Fejlesztési, Sportüze...   
39     Nif Nemzeti Infrastruktúra Fejlesztő Zártkörűe...   
...                                                  ...   
98742                              Nkm Áramhálózati Kft.   
98744                              Nkm Áramhálózati Kft.   
98751                              Nkm Áramhálózati Kft.   
98894  Bmsk Beru

In [133]:
grouped_data_300 = data[data['Szerződés típusa'] == 'Építési beruházás'].groupby('Eljárás EKR azonosító')['A beszerzés végleges összértéke'].max().reset_index()
epitesi_osszertek_300M = grouped_data_300[grouped_data_300['A beszerzés végleges összértéke'] < 300000000]
print(epitesi_osszertek_300M.shape, '\n')

epitesi_osszertek_300M_full = data[data['Eljárás EKR azonosító'].isin(epitesi_osszertek_300M['Eljárás EKR azonosító'])]
top_5_kiiro_300 = epitesi_osszertek_300M_full['Ajánlatkérő szervezet neve - tisztított'].value_counts().head(5)
top_5_nyertes_300 = epitesi_osszertek_300M_full['Nyertes ajánlattevő neve - tisztított'].value_counts().head(5)

top_5_kiiro_300 = data[(data['Ajánlatkérő szervezet neve - tisztított'].isin(top_5_kiiro_300.index)) & (data['Szerződés típusa'] == 'Építési beruházás')]
top_5_nyertes_300 = data[(data['Nyertes ajánlattevő neve - tisztított'].isin(top_5_nyertes_300.index)) & (data['Szerződés típusa'] == 'Építési beruházás')]
top_5_kiiro_300.to_csv(r'C:\Users\makos2tamas911\Documents\dev\mm\k_monitor_visu\data_exports\top_5_kiiro_300M_epitesi.csv', index=False)
top_5_nyertes_300.to_csv(r'C:\Users\makos2tamas911\Documents\dev\mm\k_monitor_visu\data_exports\top_5_nyertes_300M_epitesi.csv', index=False)

print(top_5_kiiro_300)
print('\n')
print(top_5_nyertes_300)

(11820, 2) 

      Eljárás EKR azonosító  \
212         EKR000077602018   
214         EKR000077602018   
766         EKR000462912018   
916         EKR001634822019   
925         EKR001636192019   
...                     ...   
98170       EKR001419322019   
98171       EKR001419322019   
98172       EKR001419322019   
98649       EKR001470792019   
99042       EKR001528082019   

                              Ajánlatkérő szervezet neve  \
212           Budapest Főváros Vagyonkezelő Központ Zrt.   
214           Budapest Főváros Vagyonkezelő Központ Zrt.   
766             Debrecen Megyei Jogú Város Önkormányzata   
916             Debrecen Megyei Jogú Város Önkormányzata   
925             Debrecen Megyei Jogú Város Önkormányzata   
...                                                  ...   
98170  Bdk Budapesti Dísz És Közvilágítási Korlátolt ...   
98171  Bdk Budapesti Dísz És Közvilágítási Korlátolt ...   
98172  Bdk Budapesti Dísz És Közvilágítási Korlátolt ...   
98649         

In [134]:
grouped_data_100 = data[data['Szerződés típusa'] == 'Építési beruházás'].groupby('Eljárás EKR azonosító')['A beszerzés végleges összértéke'].max().reset_index()
epitesi_osszertek_100M = grouped_data_100[grouped_data_100['A beszerzés végleges összértéke'] < 100000000]
epitesi_osszertek_100M = epitesi_osszertek_100M.sort_values('A beszerzés végleges összértéke', ascending=False)
print(epitesi_osszertek_100M.shape, '\n')

epitesi_osszertek_100M_full = data[data['Eljárás EKR azonosító'].isin(epitesi_osszertek_100M['Eljárás EKR azonosító'])]
top_5_kiiro_100 = epitesi_osszertek_100M_full['Ajánlatkérő szervezet neve - tisztított'].value_counts().head(5)
top_5_nyertes_100 = epitesi_osszertek_100M_full['Nyertes ajánlattevő neve - tisztított'].value_counts().head(5)

top_5_kiiro_100 = data[(data['Ajánlatkérő szervezet neve - tisztított'].isin(top_5_kiiro_100.index)) & (data['Szerződés típusa'] == 'Építési beruházás')]
top_5_nyertes_100 = data[(data['Nyertes ajánlattevő neve - tisztított'].isin(top_5_nyertes_100.index)) & (data['Szerződés típusa'] == 'Építési beruházás')]
top_5_kiiro_100.to_csv(r'C:\Users\makos2tamas911\Documents\dev\mm\k_monitor_visu\data_exports\top_5_kiiro_100M_epitesi.csv', index=False)
top_5_nyertes_100.to_csv(r'C:\Users\makos2tamas911\Documents\dev\mm\k_monitor_visu\data_exports\top_5_nyertes_100M_epitesi.csv', index=False)

print(top_5_kiiro_100)
print('\n')
print(top_5_nyertes_100)

(6860, 2) 

      Eljárás EKR azonosító  \
212         EKR000077602018   
214         EKR000077602018   
324         EKR001576142019   
766         EKR000462912018   
916         EKR001634822019   
...                     ...   
96212       EKR001350612019   
96352       EKR001169542019   
97029       EKR001223752019   
98649       EKR001470792019   
98943       EKR001506932019   

                              Ajánlatkérő szervezet neve  \
212           Budapest Főváros Vagyonkezelő Központ Zrt.   
214           Budapest Főváros Vagyonkezelő Központ Zrt.   
324       Székesfehérvár Megyei Jogú Város Önkormányzata   
766             Debrecen Megyei Jogú Város Önkormányzata   
916             Debrecen Megyei Jogú Város Önkormányzata   
...                                                  ...   
96212         Budapest Főváros Vagyonkezelő Központ Zrt.   
96352           Debrecen Megyei Jogú Város Önkormányzata   
97029  Budapest Főváros Ix. Kerület Ferencváros Önkor...   
98649          

In [135]:
grouped_data_900_egy_ajanlat = data[data['Beérkezett ajánlatok száma'] == 1.0].groupby('Eljárás EKR azonosító')['A beszerzés végleges összértéke'].max().reset_index()
osszertek_900M_egy_ajanlat = grouped_data_900_egy_ajanlat[grouped_data_900_egy_ajanlat['A beszerzés végleges összértéke'] > 900000000]
print(osszertek_900M_egy_ajanlat.shape, '\n')

osszertek_900M_egy_ajanlat_full = data[data['Eljárás EKR azonosító'].isin(osszertek_900M_egy_ajanlat['Eljárás EKR azonosító'])]
top_5_kiiro_900M_egy_ajanlat = osszertek_900M_egy_ajanlat_full['Ajánlatkérő szervezet neve - tisztított'].value_counts().head(5)
top_5_nyertes_900M_egy_ajanlat = osszertek_900M_egy_ajanlat_full['Nyertes ajánlattevő neve - tisztított'].value_counts().head(5)

top_5_kiiro_900M_egy_ajanlat = data[(data['Ajánlatkérő szervezet neve - tisztított'].isin(top_5_kiiro_900M_egy_ajanlat.index))]
top_5_nyertes_900M_egy_ajanlat = data[(data['Nyertes ajánlattevő neve - tisztított'].isin(top_5_nyertes_900M_egy_ajanlat.index))]
top_5_kiiro_900M_egy_ajanlat.to_csv(r'C:\Users\makos2tamas911\Documents\dev\mm\k_monitor_visu\data_exports\top_5_kiiro_900M_egy_ajanlat.csv', index=False)
top_5_nyertes_900M_egy_ajanlat.to_csv(r'C:\Users\makos2tamas911\Documents\dev\mm\k_monitor_visu\data_exports\top_5_nyertes_900M_egy_ajanlat.csv', index=False)

print(top_5_kiiro_900M_egy_ajanlat)
print('\n')
print(top_5_nyertes_900M_egy_ajanlat)

(630, 2) 

      Eljárás EKR azonosító  \
83          EKR001540992019   
93          EKR001541922019   
157         EKR001554922019   
376         EKR000458432018   
380         EKR000458432018   
...                     ...   
99130       EKR001343522019   
99131       EKR001343522019   
99132       EKR001343522019   
99133       EKR001343522019   
99134       EKR001343522019   

                              Ajánlatkérő szervezet neve  \
83               Innovációs És Technológiai Minisztérium   
93                         Nemzeti Kommunikációs Hivatal   
157    Magyar Közút Nonprofit Zártkörűen Működő Részv...   
376    Magyar Közút Nonprofit Zártkörűen Működő Részv...   
380    Magyar Közút Nonprofit Zártkörűen Működő Részv...   
...                                                  ...   
99130  Magyar Közút Nonprofit Zártkörűen Működő Részv...   
99131  Magyar Közút Nonprofit Zártkörűen Működő Részv...   
99132  Magyar Közút Nonprofit Zártkörűen Működő Részv...   
99133  Magyar Kö

In [136]:
grouped_data_300_egy_ajanlat = data[data['Beérkezett ajánlatok száma'] == 1.0].groupby('Eljárás EKR azonosító')['A beszerzés végleges összértéke'].max().reset_index()
osszertek_300M_egy_ajanlat = grouped_data_300_egy_ajanlat[grouped_data_300_egy_ajanlat['A beszerzés végleges összértéke'] < 300000000]
print(osszertek_300M_egy_ajanlat.shape, '\n')

osszertek_300M_egy_ajanlat_full = data[data['Eljárás EKR azonosító'].isin(osszertek_300M_egy_ajanlat['Eljárás EKR azonosító'])]
top_5_kiiro_300M_egy_ajanlat = osszertek_300M_egy_ajanlat_full['Ajánlatkérő szervezet neve - tisztított'].value_counts().head(5)
top_5_nyertes_300M_egy_ajanlat = osszertek_300M_egy_ajanlat_full['Nyertes ajánlattevő neve - tisztított'].value_counts().head(5)

top_5_kiiro_300M_egy_ajanlat = data[(data['Ajánlatkérő szervezet neve - tisztított'].isin(top_5_kiiro_300M_egy_ajanlat.index))]
top_5_nyertes_300M_egy_ajanlat = data[(data['Nyertes ajánlattevő neve - tisztított'].isin(top_5_nyertes_300M_egy_ajanlat.index))]
top_5_kiiro_300M_egy_ajanlat.to_csv(r'C:\Users\makos2tamas911\Documents\dev\mm\k_monitor_visu\data_exports\top_5_kiiro_300M_egy_ajanlat.csv', index=False)
top_5_nyertes_300M_egy_ajanlat.to_csv(r'C:\Users\makos2tamas911\Documents\dev\mm\k_monitor_visu\data_exports\top_5_nyertes_300M_egy_ajanlat.csv', index=False)

print(top_5_kiiro_300M_egy_ajanlat)
print('\n')
print(top_5_nyertes_300M_egy_ajanlat)

(8421, 2) 

      Eljárás EKR azonosító  \
24          EKR001533802019   
28          EKR001533802019   
30          EKR001535622019   
31          EKR001535622019   
34          EKR001535622019   
...                     ...   
98954       EKR001066492019   
98976       EKR001510892019   
98977       EKR001510892019   
99043       EKR001286872019   
99147       EKR001357862019   

                              Ajánlatkérő szervezet neve  \
24                                 Pécsi Tudományegyetem   
28                                 Pécsi Tudományegyetem   
30                                     Debreceni Egyetem   
31                                     Debreceni Egyetem   
34                                     Debreceni Egyetem   
...                                                  ...   
98954                              Pécsi Tudományegyetem   
98976  Budapesti Közlekedési Zártkörűen Működő Részvé...   
98977  Budapesti Közlekedési Zártkörűen Működő Részvé...   
99043          

In [ ]:
print('donezzzzzzzo')